In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import os
from PIL import Image
from PIL import ImageFile



In [ ]:
module_selection = ("efficientnet", 380)
handle_base, pixels = module_selection
MODULE_HANDLE = "https://tfhub.dev/tensorflow/efficientnet/lite4/feature-vector/2"
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))




In [ ]:
data_core_dir = r"D:/Betastop_dataset/insect_dataset"
BATCH_SIZE = 32
from tensorflow.keras.preprocessing.image import ImageDataGenerator
TRAINING_DIR = data_core_dir + r"/_train"
train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(380, 380))
VALIDATION_DIR= data_core_dir + r"/_validation"
validation_datagen = ImageDataGenerator(rescale=1./255,
      )
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=32,
                                                              class_mode='categorical',
                                                              target_size=(380, 380))
TEST_DIR = data_core_dir + r"/_test"
test_datagen = ImageDataGenerator(rescale=1./255,
     )
test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                              batch_size=32,
                                                              class_mode='categorical',
                                                              target_size=(380, 380))






In [ ]:
print("Using " + handle_base + ", we will build a new classifier!")
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=False),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(4,
                       activation= "softmax",
                       kernel_regularizer=keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()


In [ ]:
model.compile(optimizer = keras.optimizers.Adam(lr=0.0001),
              loss = keras.losses.categorical_crossentropy,
              metrics= ['accuracy'])



In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True ##solves PIL errors as we've resized some of the images
history = model.fit(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data=validation_generator)





In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.axis([0, 10, 0.8, 1])
plt.title('Training and validation accuracy')
plt.figure()


In [ ]:
model.evaluate(test_generator)




